In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime
import math
import threading 

## Group Assignment
### Team Number: 4
### Team Member Names: Jacky Xu, Esha Kumar, Jingyi Fang
### Team Strategy Chosen: SAFE

In [2]:
# read the ticker file
tickers = pd.read_csv("Tickers.csv",header=None)
# rename the column as "ticker"
tickers.columns=['ticker']
# create a list that contains all the tickers
ticker_lst = tickers['ticker'].tolist()


In [61]:
# call_tickers takes a list of tickers and return a list of called tickers
def call_tickers(lst):
    # create an empty list to store the called tickers
    called_tickers= []
    # iterate the ticker list
    for i in lst:
        # call each ticker and append it to the called ticker list
        called_tickers.append(yf.Ticker(i))
    # return the called ticker list when finish
    return called_tickers

# get_ticker_info takes a called tickers, an empty dataframe, an empty list 
# and store the currency and current price into the dataframe, append the called ticker into the list
def get_ticker_info(i,df,lst):
    # extract ticker name from the yf Ticker object
    ticker_name = str(i)[24:-1]
    # use try and except to ignorethe delisted stocks
    try:
        # extract currency and current price from the info dictionary and store then into the dataframe
        df.loc[ticker_name] = [i.info['currency'], i.info['currentPrice']]
        #print(ticker_name)
        # append the ticker into the ticker list
        lst.append(i)
    except:
        pass


# filter_daily_volume takes a ticker, the ticker info dataframe, the called ticker list, start date, end date, 
# if the ticker does not meet the required average daily volume, removes the ticker from the dataframe and the list 
def filter_daily_volume(i,df,lst,start,end):
    # use try and except to ignore any errors(such as there is not enough data)
    try:
        # extract the volume of the stock in the given time period 
        avg_daily_volumn = i.history(start = start, end = end,interval='1d')['Volume'].mean()
        # check if the average daily volume is < 10000
        if avg_daily_volumn < 10000:
        # if yes, remove the ticker from the list and drop the ticker from the ticker info dataframe
            lst.remove(i)
            df.drop(columns = str(i)[24:-1])
    except:
        pass


# filter_USD takes a list of called tickers and the ticker info dataframe and delete all the non-USD tickers from both
def filter_USD(lst,df):
    # Transpose the dataframe so that the ticker name becomes the index and the information becomes columns
    # filter out the USD tickers in the dataframe
    df = df[df['currency'] == 'USD']
    for i in lst:
        ticker_name = str(i)[24:-1]
        #print(ticker_name)
        # iterate the called ticker list, remove the ticker that is not in the filtered dataframe
        if ticker_name not in df.index:
            lst.remove(i)
    # return the list and the dataframe
    #df.reset_index(inplace=True)
    #df = df.drop(columns = ['index'])
    #df.columns =['ticker','currency','current_price']
    return lst, df

# get_prices takes a list of called tickers, start date adn end date, returns the historical prices in the given time period
def get_prices(lst,start,end):
    # open an empty dataframe
    prices = pd.DataFrame()
    # iterate the ticker list
    for i in lst:
        # extract the historical closing prices and store into the dataframe
        prices[str(i)[24:-1]] = i.history(start = start,end=end,interval = '1d')['Close']
    # return the prices
    return prices

# get_weekly_returns takes the historical prices(or portfolio value) and returns the weekly returns
# usually we use monthly returns but since the competition only lasts for 6 days, we decided to use the weekly returns
def get_weekly_returns(prices):
    weekly_returns = prices.resample('W').first().pct_change()
    weekly_returns = weekly_returns.iloc[1:]
    return weekly_returns

# we still use monthly returns to calculate the beta and others
def get_monthly_returns(prices):
    monthly_returns = prices.resample('MS').first().pct_change()
    #monthly_returns = monthly_returns.iloc[1:]
    return monthly_returns
    
# get_std takes the prices of the stock and returns the std of the prices
def get_std(prices):
    # open an empty dataframe with column names set up
    std = pd.DataFrame(columns = ('ticker','std'))
    # set the index
    index = 0
    # iterate through the ticker names
    for i in prices.columns:
        # store the ticker name and the std of prices
        std.loc[index] = [i, prices[i].std()]
        # add 1 to the index
        index += 1
    # return the std
    return std


# rank takes a dataframe, column name and method and rank the dataframe by the given column using the given method
def get_rank(df,column_name,method):
    # use the build-in rank function to rank the dataframe, store the rankings in a new column
    df[column_name+'_rank'] = df[column_name].rank(method=method)
    # return the ranked dataframe
    return df

# get_std_beta_rank takes the ranked std dataframe and the ranked beta dataframe, rank the dataframe by both and return the top 40 stocks
def get_std_beta_rank(std,beta):
    # combine the std dataframe and the beta dataframe
    rank = pd.concat([std,beta['beta'],beta['beta_rank']],join='inner',axis=1)
    # add up the ranks to get the final rank
    rank['final_rank'] = rank['std_rank']*8 + rank['beta_rank']
    # rank by the final rank
    rank = rank.sort_values(by='final_rank')
    # takes the top 40 stocks
    rank = rank[:40]
    # return the dataframe
    return rank



In [4]:
# set up the start date and the end date
start_date = "2021-07-02"
end_date = "2021-10-22"

# call the tickers
called_tickers = call_tickers(ticker_lst)

In [5]:
# open an empty dataframe and a list to store the ticker info and the tickers
ticker_info = pd.DataFrame(columns = ['currency','current_price'])
tickers = []
# iterate through the called tickers
for i in range(len(called_tickers)):
    # use threading to speed up the get_ticker_info function
    x = threading.Thread(target = get_ticker_info,args = (called_tickers[i],ticker_info,tickers))
    # start the thread
    x.start()


## Need to wait at least 1 minute for the threads to run, otherwise the ticker_info will be empty or missing stocks
## Same for the block below

In [199]:
# copy paste a called ticker list
tickers_copy = tickers
for i in range(len(tickers_copy)):
    x = threading.Thread(target = filter_daily_volume,args = (tickers_copy[i],ticker_info,tickers,start_date, end_date))
    x.start()

In [55]:
# filter the USD tickers
good_tickers, good_ticker_info = filter_USD(tickers, ticker_info)

# extract the closing prices for the good tickers
prices = get_prices(good_tickers, start_date, end_date)

# Get a list of Ticker Symbols (list of Strings)
ticker_symbols = prices.columns

# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [200]:
good_ticker_info.head()

,currency,current_price
PYPL,USD,188.71
KMI,USD,16.52
KO,USD,55.43
LLY,USD,262.00
COP,USD,74.83


In [57]:
# calculate the std according to the closing prices
std = get_std(prices)

# rank them in ascending order
std = get_rank(std,'std','min')

In [17]:
# Calculating Beta
def get_beta(good_tickers, prices, start_date, end_date, ticker_symbols):
    Ticker = '^GSPC'
    MarketIndex = yf.Ticker(Ticker) # The symbol yfinance uses for the S&P 500

    MarketIndex_hist = MarketIndex.history(start=start_date, end=end_date)

    # DataFrame for Market Index
    marketDF = pd.DataFrame(MarketIndex_hist['Close'])
    marketDF.columns = [Ticker]
    

    # Loop iterates through the column of prices
    betaList = []
    for i in range(len(ticker_symbols)):
        currentStockPrice = prices[ticker_symbols[i]]
        currentStockPrice = pd.concat([currentStockPrice, marketDF], join = 'inner', axis=1)
        
        # Getting the Monthly Return of each stock
        monthly_returns = currentStockPrice.resample('MS').first().pct_change()  # Dropping the first entry (since it's N/A)
        monthly_returns.drop(index=monthly_returns.index[0], inplace=True)
        
        
        # Calculate the market variance (you will need to reference the column correponding to the market)
        MarketVar = monthly_returns[Ticker].var()
        
        betaList.append(monthly_returns.cov() / MarketVar)
        
    # Filter through betaList and extract the beta for each stock
    for i in range(len(betaList)):
        betaList[i] = betaList[i].iat[0,1]
    
    # Creating a DataFrame for Tickers and their Beta value
    beta = pd.DataFrame(columns = ('ticker','beta'))
    for i in range(len(good_tickers)):
        beta.loc[i] = [str(good_tickers[i])[24:-1], betaList[i]]

    return beta

In [62]:
# get_correlation takes a ticker(self) and return 
def get_correlation(self,std_rank,prices,length):
    rank = pd.DataFrame()
    self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
    for i in range(1,length):
        candidate = std_rank.index[i]
        candidate_weekly_returns = pd.DataFrame(get_weekly_returns(prices[candidate]))
        corr = self_weekly_returns[self].corr(candidate_weekly_returns[candidate])  
        rank[candidate] = [corr]
    return rank.T

# pair_by_corr takes the ranked dataframe and the prices of the stocks, 
# returns 10 portfoliois in one dataframe, each with a pair of stocks with the lowest correlation
def pair_by_corr(rank, prices):
    # open an empty dataframe
    pairs = pd.DataFrame()
    # counter is the number of pairs
    counter = 0
    # loop until we have 10 pairs
    while counter < 10:
        # self is the stock that has lowest std and beta(which is the safest one)
        self = rank.index[0]
        # get the weekly_returns for self
        self_weekly_returns = pd.DataFrame(get_weekly_returns(prices[self]))
        # for all other stocks in the ranked dataframe
        corrs = get_correlation(self,rank,prices,len(rank)-1)
        corrs.columns = ['corr']
        corrs = corrs.sort_values(by='corr')
        champion = corrs.index[0]
        champion_weekly_returns = pd.DataFrame(get_weekly_returns(prices[champion]))
        # get current price from the ticker info dataframe
        self_current_price = good_ticker_info.loc[self,'current_price']
        champion_current_price = good_ticker_info.loc[champion,'current_price']
        # calculate the number of shares we could purchase using the current price
        self_shares = 100000 /  self_current_price
        champion_shares = 100000 /  champion_current_price
        # get weekly price, portfolio value, and portfolio return for self and champion
        pairs[self+'_price'] = prices[self].resample('W').first()
        pairs[self+'_value'] =  pairs[self+'_price']*self_shares
        pairs[self+'_return'] = pd.DataFrame(get_weekly_returns(pairs[self+'_value']))
        pairs[champion+'_price'] = prices[champion].resample('W').first()
        pairs[champion+'_value'] =  pairs[champion+'_price']*champion_shares
        pairs[champion+'_return'] = pd.DataFrame(get_weekly_returns(pairs[champion+'_value']))
        # remove self and champion from the ranked dataframe to prepare for the next iteration
        rank = rank[1:]
        rank = rank[rank.index != str(champion)]
        # add 1 to the counter for each finished pair
        counter += 1
    # return the pairs
    return pairs
            
def get_difference(df): 
    return max(abs(df['value'].max()-100000) , abs(df['value'].min()-100000))

# when 'stock' is True, create_portfolio takes two stocks and the pairs dataframe, 
# returns a weight and the weighted portfolio with the lowest sharpe ratio
# when 'stock' is False, create_portfolio takes two portfolio(old and new), the min and max range for the sharpe ratio,
# returns a weight and the weighted portfolio with the lowest sharpe ratio
def create_portfolio(old,new,range_min,range_max,stock,stock1,stock2,pairs):
    # open an empty portfolio 
    candidate = pd.DataFrame()
    champion = 0
    # min_ratio stores the lowest sharpe ratio
    min_difference = ['+' , 100000]
    # weight_1 is the weight of stock1 or the portfolio weight for old
    weight_1 = 0
    # iterates the given range
    for i in range(range_min,range_max+1):
        if stock:
            # for the stock version, the porfolio value is taken from the pairs dataframe
            # multiply the weight i with the value of each stock
            # add them up and double the result(since each stock value starts from 500000 and the required amount is 1 million)
            candidate['value'] = pairs[stock1+'_value']*(i/100)+pairs[stock2+'_value']* (1-i/100)
        else:
            # for the portfolio version, just multiply the portfolio values by the weights and add them up
            candidate['value'] = old['value']*(i/100)+new['value']* (1-i/100)
        # get the monthly returns 
        # get the sharpe ratio 
        difference = get_difference(candidate)
        #print(difference)
        # if the absolute value of the sharpe ratio(since the ratio could be negative too) is less than the min_ratio
        if abs(difference) <= min_difference[1]:
            if candidate['value'].median() > 100000:
                 min_difference = ['+',abs(difference)]
            # replace the min_ratio by the the absolute value of the current sharpe ratio 
            else:
                min_difference = ['-',abs(difference)]
            # update the champion and the weight
            champion = candidate
            weight_1 = i
    # return the optimal weight and the portfolio with the optimal weight
    return weight_1, min_difference, champion

# get_range takes the optimal weight for stock1 and returns the range for the portfolio weight
def get_range(stock1_weight):
    if (stock1_weight < 1 or stock1_weight > 99):
        range_max = 1
        range_min = 0
    # if stock1 has a greater weight, the max portfolio weight would be 35/stock1_weight
    # since 35 is the max weight, and the weight of each stock is given by the product of stock1_weight * portfolio weight
    # so the max portfolio weight is 35/stock1_weight. Same logic for the min portfolio weight
    else:
        if stock1_weight >= 50:
            range_max = 35/stock1_weight
            range_min = 2.5/(100-stock1_weight)
            # if stock1 has a lower weight, just flip the stock1 and the other stock
        else:
            range_max = 35/(100-stock1_weight) 
            range_min = 2.5/stock1_weight
    
    if range_min < range_max:
        return math.ceil(range_min*100),math.floor(range_max*100)
    else:
        return math.floor(range_max*100),math.ceil(range_min*100)


        

In [201]:
# calculate the beta according to the closing prices
beta = get_beta(good_tickers, prices, start_date, end_date, ticker_symbols)

# rank them in ascending order
beta = get_rank(beta,'beta','min')

# combine the rank of std and beta, then rank the combined rank
std_beta_rank = get_std_beta_rank(std,beta)
std_beta_rank.set_index('ticker',inplace=True)
std_beta_rank.head()

,std,std_rank,beta,beta_rank,final_rank
ticker,,,,,
T,0.663311,1.0,-0.460408,11.0,19.0
KMI,0.760351,2.0,-1.738541,5.0,21.0
KO,1.205896,4.0,1.013712,27.0,59.0
MO,1.189154,3.0,1.697517,41.0,65.0
CVS,1.819270,6.0,0.826085,23.0,71.0


In [202]:
# get the portfolio of pairs 
pairs = pair_by_corr(std_beta_rank,prices)
pairs.head()

,T_price,T_value,T_return,PEP_price,PEP_value,PEP_return,KMI_price,KMI_value,KMI_return,NEE_price,...,C_return,PG_price,PG_value,PG_return,CMCSA_price,CMCSA_value,CMCSA_return,ORCL_price,ORCL_value,ORCL_return
Date,,,,,,,,,,,,,,,,,,,,,
2021-07-04,28.158052,115071.730464,NaN,147.896271,90323.849244,NaN,18.005268,108990.726979,NaN,73.941689,...,NaN,134.218567,90285.595920,NaN,57.913876,111630.446376,NaN,81.236694,86809.889224,NaN
2021-07-11,27.955755,114245.015259,-0.007184,148.114777,90457.296086,0.001477,17.879223,108227.741343,-0.007000,74.787811,...,-0.031246,134.297577,90338.744050,0.000589,57.406128,110651.750057,-0.008767,82.487724,88146.745356,0.015400
2021-07-18,27.937721,114171.316929,-0.000645,148.492172,90687.780775,0.002548,17.956789,108697.270077,0.004338,74.747993,...,0.018033,135.443222,91109.391932,0.008531,58.003483,111803.166574,0.010406,86.459198,92390.679630,0.048146
2021-07-25,27.260860,111405.232706,-0.024227,154.739365,94503.093089,0.042071,16.657539,100832.562758,-0.072354,76.639343,...,-0.062788,138.702408,93301.767683,0.024063,56.380661,108675.136875,-0.027978,86.678474,92624.999387,0.002536
2021-08-01,27.643435,112968.673987,0.014034,156.000717,95273.431760,0.008151,17.287771,104647.525575,0.037835,76.559715,...,0.041334,139.415726,93781.599427,0.005143,58.441547,112647.547020,0.036553,87.555428,93562.115357,0.010117


In [203]:
# open empty lists for the protfolio weights, differences and symbols
portfolio_weights = []
diffs = []
stock1_symbols = []
stock2_symbols = []
# extract the column names for prices in the pairs dataframe( 'tickername_price')
symbols = pairs.filter(like='_price', axis=1).columns
# get number of pairs as the number of for loop
num_of_pairs = round(len(symbols)/2)

# then iterates the stocks in paris
for i in range(num_of_pairs):
    # get the ticker names for the pair of stocks, and append to the corresponding symbol lists
    stock1 = symbols[0][:-6]
    stock1_symbols.append(stock1)
    stock2 = symbols[1][:-6]
    stock2_symbols.append(stock2)
    
    # combine the pair of stock, store the optimal stock weight, difference, and the portfolio with optimal weight
    w1, diff, champion = create_portfolio(0,0,30,70,True,stock1,stock2,pairs)
    # append the result to their corresponding lists
    portfolio_weights.append(w1)
    diffs.append(diff)
    # remove the finished pair
    symbols = symbols[2:]


In [204]:
diffs

[['-', 2053.023320126609],
 ['-', 6431.741432167371],
 ['-', 5550.702566092747],
 ['-', 4111.608920688654],
 ['-', 11781.63673676172],
 ['-', 8429.375562358007],
 ['-', 7438.375827947966],
 ['-', 5148.881214592868],
 ['+', 5283.852196891297],
 ['+', 3509.736822636187]]

In [205]:
# format the differences with signs
for i in range(num_of_pairs):
    if diffs[i][0] == '+':
        diffs[i] = diffs[i][1]
    else:
        diffs[i] = (diffs[i][1])*-1
diffs

[-2053.023320126609,
 -6431.741432167371,
 -5550.702566092747,
 -4111.608920688654,
 -11781.63673676172,
 -8429.375562358007,
 -7438.375827947966,
 -5148.881214592868,
 5283.852196891297,
 3509.736822636187]

In [206]:
# create a dataframe with the weights and the differences
weights = pd.DataFrame(portfolio_weights, diffs)
# format the dataframe, calculate the stock 2 weight and add the symbols in
weights.reset_index(inplace=True)
weights.columns = ['diff','stock1_weight']
weights['stock1_symbol'] = stock1_symbols
weights['stock2_weight'] = 100 - weights['stock1_weight']
weights['stock2_symbol'] = stock2_symbols
weights

,diff,stock1_weight,stock1_symbol,stock2_weight,stock2_symbol
0,-2053.023320,36,T,64,PEP
1,-6431.741432,44,KMI,56,NEE
2,-5550.702566,70,KO,30,MRK
3,-4111.608921,36,MO,64,ABBV
4,-11781.636737,61,CVS,39,NKE
5,-8429.375562,30,SLB,70,ABT
6,-7438.375828,70,CSCO,30,GM
7,-5148.881215,30,USB,70,SO
8,5283.852197,70,C,30,PG
9,3509.736823,40,CMCSA,60,ORCL


In [207]:
# separate the weights dataframe by signs
pos = weights[weights['diff'] > 0]
neg = weights[weights['diff'] < 0]
# calculate the sum of positive differences and negative diffrences
pos_diff = pos['diff'].sum()
neg_diff = neg['diff'].sum()
# calculate the total differences
total_diff = pos_diff - neg_diff
# calculate the weights of positve and negative diffserences
pos_weight = pos_diff/total_diff
neg_weight = (neg_diff/total_diff)*-1
print(pos_weight, neg_weight)
# each stock must make up a minimum of (100/(2n))% of the portfolio , add/subtract 0.01 just in case
min_weight = (100/(len(weights)*4))+0.01
max_weight = 35-0.01
# the minimal weight for the entire pos/neg differences is given multiplying the minimal weight of each stock 
# and the number of stocks that has pos/neg differences, then divided by 30, since 
# 30 is the minimal weight of how each stock is weighted in its local portfolio
min_pos_weight = min_weight*len(pos)/30
min_neg_weight = min_weight*len(pos)/30
max_pos_weight = max_weight*len(neg)/30
max_neg_weight = max_weight*len(neg)/30
# use 0.5 since we wish both positive and negative differences make up of 50% of the total differences
# so that they can cancel each other
if pos_weight < neg_weight:
    # make sure that the weights is greater than the min weight
    pos_weight = min((0.5-min_neg_weight), 0.5/pos_weight)
    neg_weight = 1 - pos_weight
else:
    neg_weight = min((0.5-min_pos_weight), 0.5/neg_weight)
    pos_weight = 1 - neg_weight

pos_weight,neg_weight

0.14720029873932014 0.8527997012606798


(0.33266666666666667, 0.6673333333333333)

In [208]:
# calculate the final weight of each stock
pos['stock2_weight']*= pos_weight/len(pos)
pos['stock1_weight']*= pos_weight/len(pos)
neg['stock2_weight'] *= neg_weight/len(neg)
neg['stock1_weight'] *= neg_weight/len(neg)
neg['stock2_weight'].sum()+neg['stock1_weight'].sum()+pos['stock2_weight'].sum()+pos['stock1_weight'].sum()

<ipython-input-208-bbf606c77062>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock2_weight']*= pos_weight/len(pos)
<ipython-input-208-bbf606c77062>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos['stock1_weight']*= pos_weight/len(pos)
<ipython-input-208-bbf606c77062>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

100.0

In [211]:
# combine the pos and neg, as well as stock1 and stock2
weights = pd.concat([pos,neg],join='inner',axis=0)
weights_1 = weights[['stock1_weight','stock1_symbol']]
weights_2 = weights[['stock2_weight','stock2_symbol']]
weights_1.columns = ['weight','symbol']
weights_2.columns = ['weight','symbol']
weights = pd.concat([weights_1,weights_2],join = 'inner',axis = 0)
weights.reset_index(inplace = True)
weights.drop('index',axis = 1)

,weight,symbol
0,11.643333,C
1,6.653333,CMCSA
2,3.003000,T
3,3.670333,KMI
4,5.839167,KO
5,3.003000,MO
6,5.088417,CVS
7,2.502500,SLB
8,5.839167,CSCO
9,2.502500,USB


In [212]:
# initialize the final portfolio
final_portfolio = pd.DataFrame(columns = ['Ticker', 'Price', 'Shares', 'Value', 'Weight'])

In [213]:
# fill in the required information
for i in range(len(weights)):
    ticker = weights['symbol'][i]
    price = good_ticker_info.loc[ticker,'current_price']
    weight = weights['weight'][i]
    value = weight/100*100000
    shares = value/price
    final_portfolio.loc[i+1] = [ticker,price,shares,value,weight]

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.

In [214]:
### Check the final portfolio, using the price of start date to buy shares
ticker = final_portfolio['Ticker']
ticker_lst = ticker.tolist()
shares = final_portfolio['Shares']
price = pairs[ticker+'_price']
for i in range(len(ticker_lst)):
    ticker_name = ticker_lst[i]
    price[ticker_name+'_price'] *= final_portfolio.loc[i+1,'Shares']

price['final'] = price.sum(axis = 1)
price.tail()

<ipython-input-214-56da2f8cd03b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price[ticker_name+'_price'] *= final_portfolio.loc[i+1,'Shares']
<ipython-input-214-56da2f8cd03b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price['final'] = price.sum(axis = 1)


,C_price,CMCSA_price,T_price,KMI_price,KO_price,MO_price,CVS_price,SLB_price,CSCO_price,USB_price,...,ORCL_price,PEP_price,NEE_price,MRK_price,ABBV_price,NKE_price,ABT_price,GM_price,SO_price,final
Date,,,,,,,,,,,,,,,,,,,,,
2021-09-26,11562.993880,7212.644184,3275.675845,3429.493121,5694.846798,3279.592064,4618.370258,2171.064152,5836.535033,2310.801485,...,9157.890780,5008.382740,4375.659115,2187.710571,4729.732532,2917.013384,5807.406883,1986.628434,5856.792801,96186.004686
2021-10-03,12415.184256,7187.108162,3298.548996,3713.825755,5647.442334,3320.467900,4681.709913,2405.300106,5870.996576,2535.739375,...,9675.459989,4979.364724,4218.716011,2231.203227,4788.409797,2799.576284,5607.889251,2142.355752,5717.606930,97962.643834
2021-10-10,12248.526013,7249.670879,3280.491196,3705.077541,5582.129740,3167.865245,4576.325254,2460.273731,5701.440505,2531.355647,...,9486.286388,4897.201155,4250.854445,2527.439793,4833.306693,2782.556548,5373.025323,2172.132961,5716.691234,97172.646127
2021-10-17,12315.532595,6696.936677,3194.445936,3880.051143,5712.754928,3182.171671,4601.988951,2586.155610,5775.034692,2586.557841,...,10162.365988,5094.132828,4168.901294,2431.026079,4904.874605,2841.936907,5435.810744,2337.517694,5669.990712,98325.943181
2021-10-24,12303.505739,6825.181344,3108.540653,4039.714948,5682.205340,3278.910917,4612.909683,2721.598205,5802.369469,2533.446768,...,10292.475116,5154.450921,4338.163934,2346.473977,4833.414475,3014.971945,5453.997347,2289.230157,5718.522627,99098.506202


In [215]:
price['final']

Date
2021-07-04     96609.553480
2021-07-11     95899.055197
2021-07-18     97093.912193
2021-07-25     95508.871326
2021-08-01     97888.185054
2021-08-08     97902.808532
2021-08-15     98699.115557
2021-08-22    100390.659463
2021-08-29     99745.320172
2021-09-05     99917.498622
2021-09-12     99345.986263
2021-09-19     99020.738962
2021-09-26     96186.004686
2021-10-03     97962.643834
2021-10-10     97172.646127
2021-10-17     98325.943181
2021-10-24     99098.506202
Freq: W-SUN, Name: final, dtype: float64